## Code for collecting Reddit Data

**Code:**

```python
import praw
import pandas as pd

# Initialize Reddit API with your credentials
reddit = praw.Reddit(
    client_id="9jsUtejdbkYeYyFvxGXeXg",  
    client_secret="7q5QNOF5nrZZwNVOJy56nzs2Y36ngw",  # Your Client Secret
    user_agent="ExpediaAnalysis"  
)

# Define competitors and subreddits for search
subreddits = ["travel", "Expedia", "Airbnb", "Booking", "Vrbo", "CustomerService"]
competitor_keywords = ["Expedia", "Booking.com", "Vrbo", "Airbnb"]
limit = 1000  # **Number of posts to fetch per query**

# Fetch data
posts = []
print("Fetching Reddit posts...")

for subreddit in subreddits:
    subreddit_obj = reddit.subreddit(subreddit)
    print(f"Searching subreddit: {subreddit}")
    
    for query in competitor_keywords:
        print(f"  Searching for keyword: {query}")
        for post in subreddit_obj.search(query, limit=limit, time_filter="all"):
            posts.append({
                "Subreddit": subreddit,
                "Title": post.title[:100],  # **Truncate long titles**
                "Text": (post.selftext[:200] + '...') if len(post.selftext) > 200 else post.selftext,  # **Truncate long texts**
                "Upvotes": post.score,
                "Comments": post.num_comments,
                "URL": post.url,
                "Created": pd.to_datetime(post.created_utc, unit="s"),
                "Query": query
            })

# Save data to a DataFrame
data = pd.DataFrame(posts)

# Save to CSV for further analysis
output_path = r'C:\Users\donia\OneDrive\Desktop\HULT\Buisiness Challenge\Reddit_Competitor_Data.csv'
data.to_csv(output_path, index=False)

# Display summary of fetched posts
print("\nFetching complete!")
print(f"Total posts fetched: {len(data)}")
print(f"Data saved to: {output_path}")

In [79]:
# Import necessary libraries
import pandas as pd  # For data manipulation
from nltk.sentiment.vader import SentimentIntensityAnalyzer  # For sentiment analysis

# Initialize VADER sentiment analyzer
vader_analyzer = SentimentIntensityAnalyzer()

# Define file paths
file_path = r'C:\Users\donia\OneDrive\Desktop\Buisiness Challenge\Reddit_Competitor_Data.csv'

# Load datasets
data = pd.read_csv(file_path)  # Main Reddit competitor data
vrbo_data = pd.read_csv('VRBO_CSV.csv')  # VRBO-specific data
airbnb_data = pd.read_csv('AIRBNB_CSV.csv')  # Airbnb-specific data
expedia_data = pd.read_csv('EXPEDIA_CSV.csv')  # Expedia-specific data

# Print confirmation of dataset loading
bold = "\033[1m"  # ANSI code for bold text
reset = "\033[0m"  # ANSI code to reset text style

print(f"\n{bold}Datasets loaded successfully{reset}\n")
print(f"Main Reddit Competitor Data: {data.shape[0]} rows")
print(f"VRBO Data: {vrbo_data.shape[0]} rows")
print(f"Airbnb Data: {airbnb_data.shape[0]} rows")
print(f"Expedia Data: {expedia_data.shape[0]} rows\n")

# Define competitor keywords and their names
competitor_mapping = {
    "Expedia": "Expedia",
    "Booking.com": "Booking.com",
    "Vrbo": "Vrbo",
    "Airbnb": "Airbnb"
}

# Create a new column 'Competitor' based on mentions in Text or Title
def identify_competitor(row):
    # Check if any competitor keyword is found in the text or title of the post
    for keyword, name in competitor_mapping.items():
        if pd.notnull(row['Text']) and keyword.lower() in row['Text'].lower():
            return name
        if pd.notnull(row['Title']) and keyword.lower() in row['Title'].lower():
            return name
    return "Unknown"  # Assign 'Unknown' if no competitor is mentioned

# Apply the function to the dataset
data['Competitor'] = data.apply(identify_competitor, axis=1)

# Filter out rows without a competitor mention
data = data[data['Competitor'] != "Unknown"]

# Display the number of posts per competitor
print(f"{bold}Reddit Posts per Competitor{reset}\n")
competitor_counts = data['Competitor'].value_counts()
print(competitor_counts)

# Re-filter the data to ensure `filtered_data` is defined
filtered_data = data[data['Competitor'] != "Unknown"]

# Verify that `filtered_data` is properly defined
print(f"\n{bold}Filtered Data Rows:{reset} {len(filtered_data)}")


Datasets loaded successfully

Main Reddit Competitor Data: 2735 rows
VRBO Data: 100 rows
Airbnb Data: 100 rows
Expedia Data: 100 rows

Reddit Posts per Competitor

Competitor
Vrbo           828
Airbnb         736
Expedia        584
Booking.com    395
Name: count, dtype: int64

Filtered Data Rows: 2543


In [81]:
# Import necessary libraries
import pandas as pd  # For data manipulation
from nltk.sentiment.vader import SentimentIntensityAnalyzer  # For sentiment analysis

# Initialize VADER sentiment analyzer
vader_analyzer = SentimentIntensityAnalyzer()

# **Define ANSI escape codes for bold text**
bold = "\033[1m"
reset = "\033[0m"

# Filter posts mentioning refund and cancellation-related issues
data['RefundCancellationIssue'] = data['Text'].str.contains(
    r'\b(?:refund|refunded|non-refundable|cancel|cancelled|cancellation)\b', 
    case=False, regex=True, na=False
)

# Group data by competitor and calculate refund/cancellation-related statistics
competitor_stats = data.groupby('Competitor').agg(
    Total_Posts=('Text', 'count'),
    Refund_Cancellation_Posts=('RefundCancellationIssue', 'sum')
).reset_index()

# Calculate the percentage of refund/cancellation-related posts
competitor_stats['Refund_Cancellation_Proportion (%)'] = (
    competitor_stats['Refund_Cancellation_Posts'] / competitor_stats['Total_Posts'] * 100
)

# Format and display the results
print(f"\n{bold}Refund/Cancellation Statistics by Competitor{reset}\n")
print(competitor_stats.to_string(index=False, float_format="{:.2f}".format))

# Apply sentiment analysis
def vader_sentiment_category(text):
    if pd.isnull(text):
        return None
    scores = vader_analyzer.polarity_scores(text)
    if scores['compound'] > 0.01:  # **Positive sentiment threshold**
        return "Positive"
    elif scores['compound'] < -0.01:  # **Negative sentiment threshold**
        return "Negative"
    else:
        return "Neutral"

# Filter refund and cancellation-related posts
refund_data = data.loc[data['RefundCancellationIssue']].copy()
refund_data['SentimentCategory'] = refund_data['Text'].apply(vader_sentiment_category)

# Summarize counts and percentages
refund_sentiment_counts = refund_data.groupby(['Competitor', 'SentimentCategory'])['Text'].count().unstack(fill_value=0)
refund_sentiment_counts['Total'] = refund_sentiment_counts.sum(axis=1)
refund_sentiment_percentages = (refund_sentiment_counts.div(refund_sentiment_counts['Total'], axis=0) * 100).round(2)

# Display sentiment analysis results
print(f"\n{bold}Refund/Cancellation Issue Sentiment Analysis{reset}\n")
print(refund_sentiment_percentages)


Refund/Cancellation Statistics by Competitor

 Competitor  Total_Posts  Refund_Cancellation_Posts  Refund_Cancellation_Proportion (%)
     Airbnb          679                        224                               32.99
Booking.com          378                        153                               40.48
    Expedia          542                        277                               51.11
       Vrbo          799                        238                               29.79

Refund/Cancellation Issue Sentiment Analysis

SentimentCategory  Negative  Neutral  Positive  Total
Competitor                                           
Airbnb                47.32     1.79     50.89  100.0
Booking.com           52.29     0.65     47.06  100.0
Expedia               45.13     1.44     53.43  100.0
Vrbo                  53.36     0.00     46.64  100.0


In [51]:
# Define expanded keyword list for trust-related issues
trust_keywords = [
    # Scams and fraud
    "scam", "scams", "scammed", "fraud", "fraudulent", "fake", "sham", "phony",

    # Theft and dishonesty
    "stole", "stolen", "theft", "robbery", "stealing", "dishonest", "cheated", "lied",

    # Deceptive practices
    "deceptive", "misleading", "tricked", "false promises", "bait and switch", 
    "swindle", "ripped off", "dishonesty", "dishonest",

    # Broken trust
    "untrustworthy", "betrayed", "let down", "disappointed", "unreliable",

    # Rude behavior
    "rude", "unhelpful", "disrespectful", "arrogant", "impolite", "condescending"
]

# Filter posts containing trust-related keywords
data['TrustIssue'] = data['Text'].str.contains('|'.join(trust_keywords), case=False, na=False)

# Apply VADER sentiment analysis
def vader_sentiment_category(text):
    scores = vader_analyzer.polarity_scores(text)
    if scores['compound'] > 0.1:  # Positive threshold
        return "Positive"
    elif scores['compound'] < -0.1:  # **Negative threshold
        return "Negative"
    else:  # Neutral threshold
        return "Neutral"

# Apply sentiment classification to filtered posts
data['SentimentCategory'] = data.loc[data['TrustIssue'], 'Text'].apply(vader_sentiment_category)

# **Count positive, neutral, and negative trust-related mentions by competitor
trust_sentiment_counts = data.loc[data['TrustIssue']].groupby(
    ['Competitor', 'SentimentCategory']
)['Text'].count().unstack(fill_value=0)

# Add totals and calculate percentages
trust_sentiment_counts['Total'] = trust_sentiment_counts.sum(axis=1)
trust_sentiment_percentages = (trust_sentiment_counts.div(
    trust_sentiment_counts['Total'], axis=0) * 100).round(2)

# Display results
print(f"\n{bold}Trust Issue Percentages{reset}\n")
print(trust_sentiment_percentages)

print(f"\n{bold}Trust Issue Counts{reset}\n")
print(trust_sentiment_counts)


Trust Issue Percentages

SentimentCategory  Negative  Neutral  Positive  Total
Competitor                                           
Airbnb                51.75     1.40     46.85  100.0
Booking.com           56.20     0.83     42.98  100.0
Expedia               65.43     2.47     32.10  100.0
Vrbo                  55.41     0.00     44.59  100.0

Trust Issue Counts

SentimentCategory  Negative  Neutral  Positive  Total
Competitor                                           
Airbnb                   74        2        67    143
Booking.com              68        1        52    121
Expedia                  53        2        26     81
Vrbo                     82        0        66    148


In [53]:
# Load your dataset
data = pd.read_csv(r"C:\Users\donia\OneDrive\Desktop\Buisiness Challenge\EXPEDIA_CSV.csv")

# Group by Cancellation Options and calculate counts
grouped = data.groupby('Cancellation Options').agg(
    Total_Listings=('Price Difference', 'count'),
    No_Price_Difference=('Price Difference', lambda x: (x == 0).sum()),
    Price_Difference=('Price Difference', lambda x: (x != 0).sum())
).reset_index()

# Calculate percentages
grouped['No_Price_Difference (%)'] = (grouped['No_Price_Difference'] / grouped['Total_Listings'] * 100).round(2)
grouped['Price_Difference (%)'] = (grouped['Price_Difference'] / grouped['Total_Listings'] * 100).round(2)

# Keep relevant columns
result = grouped[['Cancellation Options', 'No_Price_Difference (%)', 'Price_Difference (%)']]

# Display the result
print(f"\n{bold}Price Difference Analysis by Cancellation Options{reset}\n")
print(result)


Price Difference Analysis by Cancellation Options

   Cancellation Options  No_Price_Difference (%)  Price_Difference (%)
0                     0                   100.00                  0.00
1                     1                    95.35                  4.65
2                     2                     6.82                 93.18
3                     3                     0.00                100.00
4                     4                     0.00                100.00


In [83]:
# Load the dataset file
file_path = r"C:\Users\donia\OneDrive\Desktop\Buisiness Challenge\VRBO_CSV.csv"
data = pd.read_csv(file_path)

# Filter out rows with missing or zero review scores
filtered_data = data[data['Review Score'] > 0]

# Calculate properties with cancellation listed prior to booking
total_properties = len(filtered_data)
listed_prior_count = filtered_data['Listed Prior to Booking'].sum()
percentage_listed_prior = (listed_prior_count / total_properties) * 100

# Analyze the impact of Cancellation Policies on Review Scores
cancellation_review_analysis = filtered_data.groupby('Cancellation').agg(
    Avg_Review_Score=('Review Score', 'mean'),
    Count=('Review Score', 'count'),
    Listed_Prior_Count=('Listed Prior to Booking', 'sum')
).reset_index().sort_values(by='Avg_Review_Score', ascending=False)

# Add percentage of listed prior to booking
cancellation_review_analysis['% Listed Prior'] = (
    (cancellation_review_analysis['Listed_Prior_Count'] / cancellation_review_analysis['Count']) * 100
).round(2)

# Drop unnecessary column
cancellation_review_analysis = cancellation_review_analysis.drop(columns=['Listed_Prior_Count'])

# Add a Total row
total_row = pd.DataFrame({
    'Cancellation': ['Total'],
    'Avg_Review_Score': ['-'],
    'Count': [total_properties],
    '% Listed Prior': [round(percentage_listed_prior, 2)]
})

cancellation_review_analysis = pd.concat([cancellation_review_analysis, total_row], ignore_index=True)

# Print the analysis results
print(f"\n{bold}VRBO: Impact of Cancellation Policies on Review Scores {reset}\n")
print(cancellation_review_analysis)


VRBO: Impact of Cancellation Policies on Review Scores 

     Cancellation Avg_Review_Score  Count  % Listed Prior
0  Non-Refundable              9.8      4          100.00
1        Moderate         9.793333     30           90.00
2         Relaxed         9.673077     52           86.54
3  Partial Refund             7.95      4          100.00
4           Total                -     90           88.89


In [73]:
# Load the dataset
file_path = r"C:\Users\donia\OneDrive\Desktop\Buisiness Challenge\AIRBNB_CSV.csv"
data = pd.read_csv(file_path)

# Filter rows with valid review scores
filtered_data = data[data['Review Score'] > 0]

# Define ANSI escape codes for bold text (if not already defined)
bold = "\033[1m"
reset = "\033[0m"


# Amount of Times Listed on Review Scores

amount_listed_stats = filtered_data.groupby('Amount of Times Listed').agg(
    Avg_Review_Score=('Review Score', 'mean'),
    Total_Reviews=('Count Reviews', 'sum'),
    Avg_Reviews_per_Property=('Count Reviews', 'mean'),
    Count_Properties=('property_id', 'count')
).reset_index()

# Display Results
print(f"\n{bold}AIRBNB: Impact of 'Amount of Times Listed' on Review Scores and Reviews:{reset}\n")
print(amount_listed_stats.to_string(index=False))


# Cancellation Policy and Amount of Times Listed

cancellation_amount_stats = filtered_data.groupby(['Amount of Times Listed', 'Cancellation']).agg(
    Avg_Review_Score=('Review Score', 'mean'),
    Total_Reviews=('Count Reviews', 'sum'),
    Count_Properties=('property_id', 'count')
).reset_index()

# Display Results
print(f"\n{bold}AIRBNB: Cancellation Policies and Amount of Times Listed:{reset}\n")
print(cancellation_amount_stats.to_string(index=False))


AIRBNB: Impact of 'Amount of Times Listed' on Review Scores and Reviews:

 Amount of Times Listed  Avg_Review_Score  Total_Reviews  Avg_Reviews_per_Property  Count_Properties
                      2          4.934433          30761                317.123711                97
                      3          4.970000            853                853.000000                 1

AIRBNB: Cancellation Policies and Amount of Times Listed:

 Amount of Times Listed Cancellation  Avg_Review_Score  Total_Reviews  Count_Properties
                      2     Flexible          4.945714           4500                14
                      2      Limited          4.937778           3688                18
                      2     Moderate          4.936053          13423                38
                      2      Relaxed          4.906429           5412                14
                      2       Strict          4.943077           3738                13
                      3     Modera

In [75]:
# Define file paths
airbnb_path = r"C:\Users\donia\OneDrive\Desktop\Buisiness Challenge\AIRBNB_CSV.csv"
vrbo_path = r"C:\Users\donia\OneDrive\Desktop\Buisiness Challenge\VRBO_CSV.csv"
expedia_path = r"C:\Users\donia\OneDrive\Desktop\Buisiness Challenge\EXPEDIA_CSV.csv"

# Load data
airbnb_data = pd.read_csv(airbnb_path)
vrbo_data = pd.read_csv(vrbo_path)
expedia_data = pd.read_csv(expedia_path)

# Define cancellation policy mappings
airbnb_policy_mapping = {
    'Relaxed': '24 Hours',
    'Flexible': '48 Hours',
    'Moderate': '5 Days',
    'Strict': '30 Days',
    'Limited': '24 Hours, then non-refundable'
}
vrbo_policy_mapping = {
    'Relaxed': '14 Days',
    'Moderate': '30 Days',
    'Partial Refund': '14 Days',
    'Non-refundable': 'No exceptions'
}

# Map cancellation policies to their timelines
airbnb_data['Cancellation Timeline'] = airbnb_data['Cancellation'].map(airbnb_policy_mapping)
vrbo_data['Cancellation Timeline'] = vrbo_data['Cancellation'].map(vrbo_policy_mapping)
expedia_data['Cancellation Timeline'] = 'Not Specified'  # Placeholder for Expedia

# Add a source column for tracking
airbnb_data['Source'] = 'Airbnb'
vrbo_data['Source'] = 'VRBO'
expedia_data['Source'] = 'Expedia'

# Manually handle non-refundable rows for VRBO
non_refundable_rows = pd.DataFrame({
    'Source': ['VRBO'] * 5,
    'Cancellation': ['Non-refundable'] * 5,
    'Cancellation Timeline': ['No exceptions'] * 5,
    'Review Score': [10, 0, 10, 9.4, 9.8],  # Review scores for these rows
    'Count Reviews': [122, 0, 471, 14, 16]  # Total reviews for these rows
})

# Append non-refundable rows to VRBO data
vrbo_data = pd.concat([vrbo_data, non_refundable_rows], ignore_index=True)

# Combine all datasets
all_data = pd.concat([airbnb_data, vrbo_data, expedia_data], ignore_index=True)

# Filter data with review scores > 0
filtered_data = all_data[all_data['Review Score'] > 0]

# Analyze the impact of cancellation timeline on review scores
cancellation_analysis = filtered_data.groupby(
    ['Source', 'Cancellation', 'Cancellation Timeline']
).agg(
    Avg_Review_Score=('Review Score', 'mean'),
    Total_Reviews=('Count Reviews', 'sum')
).reset_index()

# Sort by source and cancellation timeline
cancellation_analysis = cancellation_analysis.sort_values(
    ['Source', 'Cancellation'], ascending=[True, True]
)

# Display the final analysis
print(f"\n{bold}Impact of Cancellation Timeline on Review Scores (Including Non-refundable):{reset}\n")
print(cancellation_analysis.to_string(index=False))


Impact of Cancellation Timeline on Review Scores (Including Non-refundable):

Source   Cancellation         Cancellation Timeline  Avg_Review_Score  Total_Reviews
Airbnb       Flexible                      48 Hours          4.945714           4500
Airbnb        Limited 24 Hours, then non-refundable          4.937778           3688
Airbnb       Moderate                        5 Days          4.936923          14276
Airbnb        Relaxed                      24 Hours          4.906429           5412
Airbnb         Strict                       30 Days          4.943077           3738
  VRBO       Moderate                       30 Days          9.793333           2760
  VRBO Non-refundable                 No exceptions          9.800000            623
  VRBO Partial Refund                       14 Days          7.950000            361
  VRBO        Relaxed                       14 Days          9.673077           3255
